In [1]:
import pandas as pd
import numpy as np
import duckdb
import warnings
import os

import xgboost as xgb
from sklearn.preprocessing import LabelEncoder

# email
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

import warnings
from datetime import datetime, timedelta
from haversine import haversine

pd.set_option('display.max_columns', None)
warnings.filterwarnings("ignore")

categories = ['PTS', 'AST', 'REB', 'PR', 'PA', 'RA', 'PRA', 'TPM', 'STL', 'BLK', 'STL_BLK']
con = duckdb.connect(database=":memory:")

cwd = os.path.abspath(os.getcwd()).replace("\\", "/")
if cwd.startswith("C:/Users/Rodolfo/"):
    RUN_LOCATION = "local"
else:
    RUN_LOCATION = "cloud"
time_offset = {"local": 3, "cloud": -5}
now = str((datetime.now() + timedelta(hours=time_offset[RUN_LOCATION]) + timedelta(hours=-3)).date())
print(f"Today's date:", now)

Today's date: 2026-01-05


In [2]:
%run ./common_utils.ipynb

# Initial Functions

In [3]:
def email(model, error):
    
    # Email details
    sender_email = "rodolfoe7157@gmail.com"
    receiver_email = "rodolfoe7157@gmail.com"
    password = "cqgu bfey cnyx sfue"  # See note below

    subject = "NBA create_Predictions error"
    body = f"Model: {model}_model\nERROR: {error}"

    # Create message
    msg = MIMEMultipart()
    msg['From'] = sender_email
    msg['To'] = receiver_email
    msg['Subject'] = subject
    msg.attach(MIMEText(body, 'plain'))

    # Connect to Gmail SMTP server and send
    with smtplib.SMTP_SSL('smtp.gmail.com', 465) as server:
        server.login(sender_email, password)
        server.send_message(msg)

    print("Email sent successfully!")

In [4]:
def create_base_df():
    df = pd.DataFrame()
    df2 = pd.DataFrame()
    df3 = pd.DataFrame()
    df4 = pd.DataFrame()
    for i in [2022, 2023, 2024, 2025]:
        df_actuals = pd.read_csv(f"../tables/{i}/parlay_stats.csv")
        df_actuals['Season'] = i
        df = pd.concat([df, df_actuals])

        df_schd = pd.read_csv(f"../tables/{i}/nba_schedule.csv")
        df_schd['Season'] = i
        df2 = pd.concat([df2, df_schd])

        df_gms = pd.read_csv(f"../tables/{i}/season_gamelogs.csv")
        df_gms['Season'] = i
        df3 = pd.concat([df3, df_gms])

        df_inj = pd.read_csv(f"../tables/{i}/injuries.csv")
        df_inj['Season'] = i
        df4 = pd.concat([df4, df_inj])

    df['Date'] = pd.to_datetime(df.Date)
    df2['Date'] = pd.to_datetime(df2.Date)
    df3['Date'] = pd.to_datetime(df3.Date)
    df3 = df3[~df3[['Date', 'Team', 'Player']].duplicated(keep='last')]
    df4['Date'] = pd.to_datetime(df4.Date)

    df3_temp = df3.rename(columns={"3PM": "TPM", "3PA": "TPA", "3P%": "TP%", "TRB": "REB"}).drop(['Pos', 'Opp'], axis=1)
    df3_temp['PR'] = df3_temp.PTS + df3_temp.REB 
    df3_temp['PA'] = df3_temp.PTS + df3_temp.AST
    df3_temp['RA'] = df3_temp.REB + df3_temp.AST
    df3_temp['PRA'] = df3_temp.PTS + df3_temp.REB + df3_temp.AST
    df3_temp['STL_BLK'] = df3_temp.STL + df3_temp.BLK
    df = df.merge(df3_temp, on=['Season', 'Date', 'Team', 'Player'], how='left')

    df_mtch = df2[['Season', 'Date', 'AwayABV', 'HomeABV', 'AwayPTS', 'HomePTS', 'AwayB2B', 'HomeB2B', 'is_OT', 'cup_gm', 'pstszn_gm']]
    df_mtch['Team_type'] = 'Away'
    df_mtch = df_mtch.rename(columns={"AwayABV": "Team", "HomeABV": "Opp", "AwayB2B": "B2B"})[['Season', 'Date', 'Team', 'AwayPTS', 'HomePTS', 'Opp', 'B2B', 'is_OT', 'cup_gm', 'pstszn_gm', 'Team_type']]
    df_mtch2 = df_mtch.copy().rename(columns={"Team": "Opp", "Opp": "Team", "HomeB2B": "B2B"})[['Season', 'Date', 'Team', 'AwayPTS', 'HomePTS', 'Opp', 'B2B', 'is_OT', 'cup_gm', 'pstszn_gm']]
    df_mtch2['Team_type'] = 'Home'
    df_mtch = pd.concat([df_mtch, df_mtch2])
    df_mtch = df_mtch[['Season', 'Date', 'Team', 'Team_type', 'AwayPTS', 'HomePTS', 'is_OT', 'cup_gm', 'pstszn_gm']]
    df_mtch = df_mtch.sort_values(["Team", "Date"])
    df_mtch['team_game_num'] = df_mtch.groupby(["Team", "Season"]).cumcount() + 1
    df_mtch['Spread'] = np.where(df_mtch.Team_type == 'Home', df_mtch.HomePTS - df_mtch.AwayPTS, df_mtch.AwayPTS - df_mtch.HomePTS)
    df_mtch['Total'] = df_mtch.AwayPTS + df_mtch.HomePTS
    df_mtch['is_Win'] = np.where(df_mtch.Spread > 0, 1, 0)
    df_mtch['Szn_Wins'] = df_mtch.groupby(['Season', 'Team'])['is_Win'].cumsum()
    df = df.drop(['Season', 'Team_type'], axis=1).merge(df_mtch, on=['Date', 'Team'])

    df = df.merge(df4[['Date', 'Team', 'Player', 'Status']], on=['Date', 'Team', 'Player'], how='left')
    df['Status'] = np.where((df.Active == 1) & (df.Status.isnull()), 'Available', df.Status)
    df['Status'] = np.where((df.Active == 0), 'Out', df.Status)
    df['Status'] = np.where((df.Status == 'Out') & (df.Active != 0), 'Available', df.Status)
    
    return df

# Minutes Projection Model

In [5]:
def setup_df_mins(con, df):
    
    df = df[['Season', 'Date', 'Team', 'Team_type', 'Opp', 'Player', 'Pos', 'B2B', 'MP',
             'Spread', 'Total', 'team_game_num', 'Szn_Wins', 'pstszn_gm', 'is_OT']]
    
    for col in ['MP']:
        for N in [1, 3, 5, 10]:
            df[f'{col}_L{N}_avg'] = (
                df.groupby(['Player', 'Season'])[col]
                  .rolling(window=N, min_periods=1)
                  .mean()
                  .shift(1)
                  .reset_index(level=[0, 1], drop=True)
            )

    games_last_7_days = df.sort_values(['Player', 'Season', 'Date']).groupby(['Player', 'Season']).rolling('7D', on='Date', closed='left')['MP'].count().reset_index().rename(columns={"MP": "gms_L7_days"})
    games_last_7_days = games_last_7_days.drop_duplicates(
        subset=['Player', 'Season', 'Date']
    )
    df = df.merge(games_last_7_days, on=['Player', 'Season', 'Date'])
    df['gms_L7_days'] = df.gms_L7_days.fillna(0).astype(int)
    
    df['prev_team_mins_pct'] = (df.groupby(['Player', 'Season'])['MP'].shift(1)) / 240
           
    df['reserve_td'] = (df.MP < 8).astype(int)
    df['bench_td']   = ((df.MP >= 8) & (df.MP <= 25)).astype(int)
    df['starter_td'] = (df.MP > 25).astype(int)
    role_counts = df.groupby(['Season', 'Player'])[['reserve_td', 'bench_td', 'starter_td']].sum()
    role_counts['most_common_role'] = role_counts[['reserve_td', 'bench_td', 'starter_td']].idxmax(axis=1)
    role_counts['reserve'] = (role_counts['most_common_role'] == 'reserve_td').astype(int)
    role_counts['bench']   = (role_counts['most_common_role'] == 'bench_td').astype(int)
    role_counts['starter'] = (role_counts['most_common_role'] == 'starter_td').astype(int)
    df = df.merge(role_counts[['reserve', 'bench', 'starter']], on=['Season', 'Player'], how='left')
    
    df['role'] = 0
    df['role'] = np.where(df.starter == 1, 1, df.role)
    df['role'] = np.where(df.bench == 1, 2, df.role)
    df['role'] = np.where(df.reserve == 1, 3, df.role)
    
    for N in [1, 3, 5]:
        for role in ['reserve_td', 'bench_td', 'starter_td']:
            df[f'{role}_last{N}'] = (
                df.sort_values(['Player', 'Date']).groupby('Player')[role]
                  .rolling(N, min_periods=1)
                  .sum()
                  .shift(1)
                  .reset_index(0, drop=True)
            )
        rec_role_cols = [f'{role}_last{N}' for role in ['reserve_td', 'bench_td', 'starter_td']]
        df[f'recent_most_common_role_L{N}'] = df[rec_role_cols].idxmax(axis=1)
        df[f'recent_role_L{N}'] = 0
        df[f'recent_role_L{N}'] = np.where(df[f'recent_most_common_role_L{N}'] == f'starter_td_last{N}', 1, df[f'recent_role_L{N}'])
        df[f'recent_role_L{N}'] = np.where(df[f'recent_most_common_role_L{N}'] == f'bench_td_last{N}', 2, df[f'recent_role_L{N}'])
        df[f'recent_role_L{N}'] = np.where(df[f'recent_most_common_role_L{N}'] == f'reserve_td_last{N}', 3, df[f'recent_role_L{N}'])      
        df = df.drop(f'recent_most_common_role_L{N}', axis=1)
        for role in ['reserve_td', 'bench_td', 'starter_td']:
            df = df.drop(f'{role}_last{N}', axis=1)
      
    df['missed_games'] = (
        df.groupby(['Player', 'Team', 'Season'])['team_game_num']      
          .diff()
          .sub(1)
          .fillna(0)
          .astype(int)
    )
    
    df['game_spread_type'] = 0
    df['game_spread_type'] = np.where(abs(df.Spread < 6), 1, df.game_spread_type) 
    df['game_spread_type'] = np.where((abs(df.Spread >= 6) & abs(df.Spread <= 14)), 2, df.game_spread_type) 
    df['game_spread_type'] = np.where(abs(df.Spread > 14), 3, df.game_spread_type) 
    
    df['missed_games_after'] = (
        df.groupby(['Player', 'Team', 'Season'])['team_game_num']
          .diff(-1)
          .abs()
          .sub(1)
          .fillna(0)
          .astype(int)
    )
    df['MP_diff_next_avg5'] = df.groupby('Player')['MP'].shift(-1) - df['MP_L5_avg']
    df['Injured'] = (
                        ((df.missed_games_after > 0) & (df.MP_diff_next_avg5 < -10) & (df.role == 1)) | 
                        ((df.missed_games_after > 0) & (df.MP_diff_next_avg5 < -5) & (df.role == 2))
                    ).astype(int)
    
    df['return_game'] = (df.groupby('Player')['Injured'].shift(1) == 1).astype(int)
    df['games_since_return'] = (df.groupby('Player')['return_game'].cumsum())
    df['games_since_return'] = (df.groupby(['Player', 'games_since_return']).cumcount())
    df['ramp_phase'] = 0
    df.loc[df.return_game == 1, 'ramp_phase'] = 1
    df.loc[df.games_since_return.isin([1, 2]), 'ramp_phase'] = 2
    df.loc[df.games_since_return >= 3, 'ramp_phase'] = 3
    df['starter_return'] = ((df.return_game == 1) & (df.role == 1)).astype(int)
    df['bench_return']   = ((df.return_game == 1) & (df.role == 2)).astype(int)
    
    df['MP_change_pct_L5'] = (df['MP_L1_avg'] - df['MP_L10_avg']) / df['MP_L10_avg']
    df['Early_stop'] = (
        (df['MP'] < 5) |
        ((df.role == 1) & (df.MP_change_pct_L5 <= -0.35)) |   # starters
        ((df.role == 2) & (df.MP_change_pct_L5 <= -0.45)) |   # bench
        ((df.role == 3) & (df.MP_change_pct_L5 <= -0.55))     # reserve
    ).astype(int)
    df['Early_stop'] = df.groupby('Player')['Early_stop'].shift(-1).fillna(0).astype(int)
    df['MP_increase'] = (
        ((df.role == 1) & (df.MP_change_pct_L5 >= 0.15)) |
        ((df.role == 2) & (df.MP_change_pct_L5 >= 0.10)) |
        ((df.role == 3) & (df.MP_change_pct_L5 >= 0.05))
    ).astype(int)
    df['MP_increase'] = df.groupby('Player')['MP_increase'].shift(-1).fillna(0).astype(int)
    
    for N in [1, 3, 5]:
        df[f'Early_stop_L{N}'] = df.sort_values(['Player', 'Date'])['Early_stop'].shift(1).rolling(N).sum()
        df[f'Injured_L{N}'] = df.sort_values(['Player', 'Date'])['Injured'].shift(1).rolling(N).sum()
    
    # Location based features
    df["DaysLstGm"] = (df.groupby("Player")["Date"].diff().dt.days).fillna(0).astype(int)
    df['Location'] = df.apply(lambda r: r['Team'] if r['Team_type'] == 'Home' else r['Opp'], axis=1)
    df['PrevLocation'] = df.groupby('Player')['Location'].shift(1)
    df['same_arena'] = (df['PrevLocation'] == df['Location']).astype(int)

    df = df.drop(['Season', 'Team_type', 'reserve_td', 'reserve', 'bench_td', 'bench', 'starter_td', 'starter', 
                  'PrevLocation', 'Location', 'Injured', 'missed_games_after', 'MP_diff_next_avg5', 'return_game'], axis=1)    
    
    return df

# Main Model

In [6]:
def setup_df_main(df, tgt_stat):
    
    # Stat dependent features 
    if tgt_stat == 'PTS':
        tgt_stat_cols = ['TPM', 'FG', 'FT', 'TPA', 'FGA', 'FTA']
        df = df[['Season', 'Date', 'Team', 'Opp', 'Player', 'Pos', 'MP', 'team_game_num', 
         'PTS', 'TPM', 'FG', 'FGA', 'TPA', 'FT', 'FTA', 
         f'Off_{tgt_stat}', f'Off_L3_{tgt_stat}', f'Off_L5_{tgt_stat}', f'Off_L10_{tgt_stat}', f'Off_{tgt_stat}_Rk',
         f'Def_{tgt_stat}', f'Def_L3_{tgt_stat}', f'Def_L5_{tgt_stat}', f'Def_L10_{tgt_stat}', f'Def_{tgt_stat}_Rk',
         'Spread', 'Total', 'is_OT']]
        
        # Efficiency metrics
        df['three_rate_raw'] =  np.where(df.FGA > 0, df['TPA'] / df['FGA'], 0)
        df['ft_rate_raw']    =  np.where(df.FGA > 0, df['FTA'] / df['FGA'], 0)
        df['eFG_raw'] = (df['FG'] + 0.5 * df['TPM']) / df['FGA']
        df['TS_raw'] = df['PTS'] / (2 * (df['FGA'] + 0.44 * df['FTA']))    
        df['usage_proxy_raw'] =  np.where(df.MP > 0, (df['FGA'] + 0.44 * df['FTA']) / df['MP'], 0)
        
        for w in [3, 5, 10]:
            for metric in ['three_rate', 'ft_rate', 'eFG', 'TS', 'usage_proxy']:
                col = f"{metric}_L{w}"
                df[col] = (
                    df.groupby(['Player','Season'])[f'{metric}_raw']
                      .rolling(w, min_periods=1)
                      .mean()
                      .shift(1)
                      .reset_index(level=[0,1], drop=True)
                )
        for metric in ['three_rate', 'ft_rate', 'eFG', 'TS', 'usage_proxy']:
            col = f'{metric}_weighted'
            df[col] = (
                0.6 * df[f'{metric}_L3'] +
                0.3 * df[f'{metric}_L5'] +
                0.1 * df[f'{metric}_L10']
            )
            df = df.drop(f'{metric}_raw', axis=1)
        
    elif tgt_stat == 'PRA':
        tgt_stat_cols = ['PTS', 'REB', 'AST', 'TPM', 'FG']
        df = df[['Season', 'Date', 'Team', 'Opp', 'Player', 'Pos', 'MP', 'team_game_num', 
         'PTS', 'AST', 'REB', 'PR', 'PA', 'RA', 'PRA', 'TPM', 'STL', 'BLK', 'STL_BLK', 
         'FG', 'FGA', 'TPA', 'FT', 'FTA', 
         f'Off_{tgt_stat}', f'Off_L3_{tgt_stat}', f'Off_L5_{tgt_stat}', f'Off_L10_{tgt_stat}', f'Off_{tgt_stat}_Rk',
         f'Def_{tgt_stat}', f'Def_L3_{tgt_stat}', f'Def_L5_{tgt_stat}', f'Def_L10_{tgt_stat}', f'Def_{tgt_stat}_Rk',
         'Spread', 'Total', 'is_OT']]
        
        df['usage_proxy_raw'] =  np.where(df.MP > 0, (df['FGA'] + 0.44 * df['FTA']) / df['MP'], 0)
        for w in [3, 5, 10]:
            df[f"usage_proxy_L{w}"] = (
                df.groupby(['Player','Season'])[f'usage_proxy_raw']
                  .rolling(w, min_periods=1)
                  .mean()
                  .shift(1)
                  .reset_index(level=[0,1], drop=True)
            )
        df['usage_proxy_weighted'] = (
            0.6 * df[f'usage_proxy_L3'] +
            0.3 * df[f'usage_proxy_L5'] +
            0.1 * df[f'usage_proxy_L10']
        )
        df = df.drop('usage_proxy_raw', axis=1)
        
        
    else:
        tgt_stat_cols = []
        df = df[['Season', 'Date', 'Team', 'Opp', 'Player', 'Pos', 'MP', 'team_game_num', 
         'PTS', 'AST', 'REB', 'PR', 'PA', 'RA', 'PRA', 'TPM', 'STL', 'BLK', 'STL_BLK',
         'FG', 'FGA', 'TPA', 'FT', 'FTA', 
          f'Off_{tgt_stat}', f'Off_L3_{tgt_stat}', f'Off_L5_{tgt_stat}', f'Off_L10_{tgt_stat}', f'Off_{tgt_stat}_Rk',
          f'Def_{tgt_stat}', f'Def_L3_{tgt_stat}', f'Def_L5_{tgt_stat}', f'Def_L10_{tgt_stat}', f'Def_{tgt_stat}_Rk',
         'Spread', 'Total', 'is_OT']]

    
    # Create rolling + lag features    
    for col in ['MP'] + tgt_stat_cols:
        df[f'{col}_lst_gm'] = (
            df
            .groupby(['Player', 'Season'])[col]
            .shift(1)
        )
        for N in [1, 3, 5, 10]:
            df[f'{col}_L{N}_avg'] = (
                df.groupby(['Player', 'Season'])[col]
                  .rolling(window=N, min_periods=1)
                  .mean()
                  .shift(1)
                  .reset_index(level=[0, 1], drop=True)
            )

    # Role identifiers features
    df['reserve_td'] = (df.MP < 8).astype(int)
    df['bench_td']   = ((df.MP >= 8) & (df.MP <= 25)).astype(int)
    df['starter_td'] = (df.MP > 25).astype(int)
    role_counts = df.groupby(['Season', 'Player'])[['reserve_td', 'bench_td', 'starter_td']].sum()
    role_counts['most_common_role'] = role_counts[['reserve_td', 'bench_td', 'starter_td']].idxmax(axis=1)
    role_counts['reserve'] = (role_counts['most_common_role'] == 'reserve_td').astype(int)
    role_counts['bench']   = (role_counts['most_common_role'] == 'bench_td').astype(int)
    role_counts['starter'] = (role_counts['most_common_role'] == 'starter_td').astype(int)
    df = df.merge(role_counts[['reserve', 'bench', 'starter']], on=['Season', 'Player'], how='left')
    df['role'] = 0
    df['role'] = np.where(df.starter == 1, 1, df.role)
    df['role'] = np.where(df.bench == 1, 2, df.role)
    df['role'] = np.where(df.reserve == 1, 3, df.role)
    
    for N in [1, 3, 5]:
        for role in ['reserve_td', 'bench_td', 'starter_td']:
            df[f'{role}_last{N}'] = (
                df.sort_values(['Player', 'Date']).groupby('Player')[role]
                  .rolling(N, min_periods=1)
                  .sum()
                  .shift(1)
                  .reset_index(0, drop=True)
            )
        rec_role_cols = [f'{role}_last{N}' for role in ['reserve_td', 'bench_td', 'starter_td']]
        df[f'recent_most_common_role_L{N}'] = df[rec_role_cols].idxmax(axis=1)
        df[f'recent_role_L{N}'] = 0
        df[f'recent_role_L{N}'] = np.where(df[f'recent_most_common_role_L{N}'] == f'starter_td_last{N}', 1, df[f'recent_role_L{N}'])
        df[f'recent_role_L{N}'] = np.where(df[f'recent_most_common_role_L{N}'] == f'bench_td_last{N}', 2, df[f'recent_role_L{N}'])
        df[f'recent_role_L{N}'] = np.where(df[f'recent_most_common_role_L{N}'] == f'reserve_td_last{N}', 3, df[f'recent_role_L{N}'])      
        df = df.drop(f'recent_most_common_role_L{N}', axis=1)
        for role in ['reserve_td', 'bench_td', 'starter_td']:
            df = df.drop(f'{role}_last{N}', axis=1)
    
    df['game_spread_type'] = 0
    df['game_spread_type'] = np.where(abs(df.Spread < 6), 1, df.game_spread_type) 
    df['game_spread_type'] = np.where((abs(df.Spread >= 6) & abs(df.Spread <= 14)), 2, df.game_spread_type) 
    df['game_spread_type'] = np.where(abs(df.Spread > 14), 3, df.game_spread_type) 
    
    for col in categories + ['Season', 'FG', 'FGA', 'FT', 'FTA', 'TPM', 'TPA', 
                             'reserve_td', 'reserve', 'bench_td', 'bench', 'starter_td', 'starter'] + tgt_stat_cols:
        if col == tgt_stat:
            continue
        if col in df.columns:
            df = df.drop(col, axis=1)
        
    return df

### Today's predictions

In [7]:
def generate_predictions(tgt_stat):
    team_encoder = LabelEncoder()
    player_encoder = LabelEncoder()
    team_type_encoder = LabelEncoder()
    position_encoder = LabelEncoder()
    status_encoder = LabelEncoder()
    
    df_pred = create_base_df()
    
    # Encode string cols
    team_encoder.fit(pd.concat([df_pred["Team"], df_pred["Opp"]], axis=0))
    df_pred["Team"] = team_encoder.transform(df_pred["Team"])
    df_pred["Opp"] = team_encoder.transform(df_pred["Opp"])
    df_pred["Player"] = player_encoder.fit_transform(df_pred["Player"])
    df_pred["Pos"] = position_encoder.fit_transform(df_pred["Pos"])
    df_pred['Team_type'] = team_type_encoder.fit_transform(df_pred['Team_type'])
    df_pred["Status"] = status_encoder.fit_transform(df_pred["Status"])
    
    mins_model = xgb.XGBRegressor()
    mins_model.load_model("../ML_models/mins_model.json")
    stat_model = xgb.XGBRegressor()
    stat_model.load_model(f"../ML_models/{tgt_stat}_model.json")
    
    df_lines = pd.read_csv(f"../tables/2025/parlay_lines.csv")
    df_lines['Date'] = pd.to_datetime(df_lines.Date)
    df_lines = df_lines[~(df_lines.Team.isnull())]

    # Predict Mins
    df_lines["Team"] = team_encoder.transform(df_lines["Team"])
    df_pred = df_pred.merge(df_lines[['Date', 'Team', 'Spread', 'Total']], on=['Date', 'Team'], how='left')
    df_pred = df_pred[~df_pred[['Date', 'Team', 'Player']].duplicated(keep='last')]
    df_pred['Spread_x'] = np.where(df_pred.Spread_x.isnull(), df_pred.Spread_y, df_pred.Spread_x)
    df_pred['Total_x'] = np.where(df_pred.Total_x.isnull(), df_pred.Total_y, df_pred.Total_x)
    df_pred = df_pred.rename(columns={"Spread_x": "Spread", "Total_x": "Total"}).drop(['Spread_y', 'Total_y'], axis=1)
    df_pred_mins = setup_df_mins(con, df_pred)
    df_pred_mins = df_pred_mins.drop(['Date', 'MP'], axis=1)
    df_pred['MP'] = mins_model.predict(df_pred_mins)

    # Predict Stat
    df_pred = setup_df_main(df_pred, tgt_stat)
    feature_cols = [col for col in df_pred.columns if col not in ['Date', tgt_stat]]
    df_pred = df_pred[df_pred.Date == now][feature_cols]
    df_pred[f"{tgt_stat}_proj"] = stat_model.predict(df_pred)

    df_pred['Team'] = team_encoder.inverse_transform(df_pred["Team"])
    df_lines['Team'] = team_encoder.inverse_transform(df_lines["Team"])
    df_pred['Opp'] = team_encoder.inverse_transform(df_pred["Opp"])
    df_pred['Player'] = player_encoder.inverse_transform(df_pred["Player"])
    df_pred['Pos'] = position_encoder.inverse_transform(df_pred["Pos"])

    df_lines = df_lines[df_lines.Date == now][['Team', 'Player', f'{tgt_stat}_line']]
    df_pred = df_pred.merge(df_lines, on=['Team', 'Player'])

    tds_picks = df_pred[~(df_pred[f'{tgt_stat}_line'].isnull())]\
                [['Team', 'Player', 'Pos', 'Opp', 'MP', 'MP_L5_avg', f'{tgt_stat}_line', f'{tgt_stat}_proj']]
    tds_picks['Diff'] = abs((df_pred[f'{tgt_stat}_line'] - df_pred[f'{tgt_stat}_proj']))
    tds_picks['Diff2'] = abs((df_pred['MP'] - df_pred['MP_L5_avg']))
    tds_picks = tds_picks.sort_values('Diff', ascending=False).drop(['Diff', 'Diff2'], axis=1)
    if tds_picks.shape[0] >= 50:
        print(tds_picks.shape[0], 'rows')
        for tm in tds_picks.Team.unique():
            display(tds_picks[tds_picks.Team == tm])
    else:
        display(tds_picks)
    tds_picks.insert(0, 'Date', pd.to_datetime(now))
    partition_save_df(tds_picks, f"../tables/2025/gmday_preds_{tgt_stat}.csv")

In [8]:
try: 
    generate_predictions('PTS')
except Exception as e:
    email('PTS', e)
    raise Exception(e)

108 rows


,Team,Player,Pos,Opp,MP,MP_L5_avg,PTS_line,PTS_proj
46,DEN,Julian Strawther,SG,PHI,4.682094,8.022773,14.5,4.591036
45,DEN,Jalen Pickett,SG,PHI,12.365000,14.703943,11.5,5.007154
39,DEN,Bruce Brown,SG,PHI,24.777317,26.453680,11.5,5.473209
62,DEN,Peyton Watson,SF,PHI,30.415960,35.411565,19.5,17.928331


,Team,Player,Pos,Opp,MP,MP_L5_avg,PTS_line,PTS_proj
2,UTA,Keyonte George,PG,POR,36.956394,35.587633,25.5,33.022549
72,UTA,Lauri Markkanen,PF,POR,36.715687,34.865586,26.5,30.849905
68,UTA,Brice Sensabaugh,SF,POR,24.702726,28.107692,13.5,11.254601
99,UTA,Kyle Filipowski,C,POR,19.174109,20.504084,8.5,6.409704
98,UTA,Jusuf Nurkic,C,POR,27.059151,29.803619,11.5,13.070454
67,UTA,Ace Bailey,SF,POR,21.282032,24.343173,10.5,8.960531
69,UTA,Svi Mykhailiuk,SF,POR,15.497404,17.003059,6.5,5.224282


,Team,Player,Pos,Opp,MP,MP_L5_avg,PTS_line,PTS_proj
105,DET,Paul Reed,C,NYK,8.919422,10.574579,10.5,3.710186
60,DET,Ausar Thompson,SF,NYK,22.790836,25.124680,12.5,8.516575
59,DET,Duncan Robinson,SF,NYK,25.317282,27.087841,11.5,7.992232
5,DET,Cade Cunningham,PG,NYK,36.603371,34.571375,27.5,30.468531
35,DET,Jaden Ivey,SG,NYK,16.470301,18.164440,9.5,6.924878
101,DET,Isaiah Stewart,C,NYK,22.498543,26.096553,11.5,9.042394
40,DET,Javonte Green,SG,NYK,19.103491,23.069702,7.5,6.789832


,Team,Player,Pos,Opp,MP,MP_L5_avg,PTS_line,PTS_proj
58,CHO,Brandon Miller,SF,OKC,33.418228,32.724183,17.5,24.103287
61,CHO,Kon Knueppel,SF,OKC,31.403574,28.871992,16.5,22.389772
20,CHO,Collin Sexton,SG,OKC,20.183510,22.418981,11.5,14.005904
7,CHO,LaMelo Ball,PG,OKC,26.490702,25.446436,17.5,19.922857
86,CHO,Tidjane Salaun,PF,OKC,14.505600,19.261573,5.5,7.639359
102,CHO,Moussa Diabate,C,OKC,17.360538,26.887075,7.5,5.953083
76,CHO,Miles Bridges,PF,OKC,30.541817,30.184010,16.5,15.583406


...

,Team,Player,Pos,Opp,MP,MP_L5_avg,PTS_line,PTS_proj
83,TOR,Collin Murray-Boyles,PF,ATL,18.676691,23.619989,8.5,5.192416
97,TOR,Sandro Mamukelashvili,C,ATL,22.746017,23.528446,11.5,8.504154
49,TOR,Brandon Ingram,SF,ATL,36.023071,34.390680,23.5,26.479996
54,TOR,RJ Barrett,SF,ATL,26.570839,27.547768,18.5,16.095015
14,TOR,Jamal Shead,PG,ATL,18.583153,24.783638,6.5,7.255374
74,TOR,Scottie Barnes,PF,ATL,35.425617,35.956332,18.5,17.957691
9,TOR,Immanuel Quickley,PG,ATL,32.822514,33.526613,15.5,16.041466
44,TOR,Ja'Kobe Walter,SG,ATL,18.360394,24.024553,5.5,5.414319


,Team,Player,Pos,Opp,MP,MP_L5_avg,PTS_line,PTS_proj
21,PHI,VJ Edgecombe,SG,DEN,36.161594,35.010011,16.5,19.495584
6,PHI,Tyrese Maxey,PG,DEN,36.764439,37.884894,26.5,29.494631
25,PHI,Quentin Grimes,SG,DEN,30.096149,33.341034,11.5,10.489440
73,PHI,Paul George,PF,DEN,32.970432,33.810214,14.5,15.035506
88,PHI,Joel Embiid,C,DEN,31.255186,31.306263,25.5,25.592812


,Team,Player,Pos,Opp,MP,MP_L5_avg,PTS_line,PTS_proj
18,POR,Shaedon Sharpe,SG,UTA,31.132620,30.755154,23.5,20.958132
75,POR,Toumani Camara,PF,UTA,34.966282,32.585009,14.5,12.001682
96,POR,Donovan Clingan,C,UTA,29.857204,27.847939,12.5,13.130460
70,POR,Kris Murray,SF,UTA,24.163584,17.630992,7.5,7.816152
50,POR,Deni Avdija,SF,UTA,37.968773,36.253470,28.5,28.762779


,Team,Player,Pos,Opp,MP,MP_L5_avg,PTS_line,PTS_proj
28,GSW,Moses Moody,SG,LAC,21.217152,23.146505,9.5,7.502432
16,GSW,De'Anthony Melton,PG,LAC,18.365576,24.555889,8.5,6.700829
23,GSW,Brandin Podziemski,SG,LAC,25.675646,25.722337,10.5,11.589371
8,GSW,Stephen Curry,PG,LAC,34.862770,33.953576,27.5,28.392885
85,GSW,Quinten Post,PF,LAC,17.151863,17.411929,7.5,6.880451
104,GSW,Al Horford,C,LAC,14.764151,14.767115,5.5,5.728303
48,GSW,Jimmy Butler,SF,LAC,33.458622,31.185701,19.5,19.273588
84,GSW,Draymond Green,PF,LAC,22.098801,24.962959,7.5,7.603770


../tables/2025/gmday_preds_PTS.csv saved!


In [9]:
try: 
    generate_predictions('PRA')
except Exception as e:
    email('PRA', e)
    raise Exception(e)

82 rows


,Team,Player,Pos,Opp,MP,MP_L5_avg,PRA_line,PRA_proj
45,DEN,Jalen Pickett,SG,PHI,12.365000,14.703943,21.5,7.946320
39,DEN,Bruce Brown,SG,PHI,24.777317,26.453680,21.5,13.754008
62,DEN,Peyton Watson,SF,PHI,30.415960,35.411565,27.5,23.260921


,Team,Player,Pos,Opp,MP,MP_L5_avg,PRA_line,PRA_proj
26,CHI,Coby White,SG,BOS,23.570511,25.730948,25.5,16.074398
89,CHI,Nikola Vucevic,C,BOS,32.138180,28.086426,30.5,33.856998
32,CHI,Ayo Dosunmu,SG,BOS,22.001638,26.488601,19.5,17.294138
42,CHI,Isaac Okoro,SG,BOS,22.057320,25.541365,13.5,12.334515
79,CHI,Matas Buzelis,PF,BOS,26.319359,26.904252,23.5,24.449713
63,CHI,Kevin Huerter,SF,BOS,22.609478,23.925695,18.5,18.729156


,Team,Player,Pos,Opp,MP,MP_L5_avg,PRA_line,PRA_proj
0,OKC,Shai Gilgeous-Alexander,PG,CHO,32.054302,31.283890,42.5,34.512955
36,OKC,Aaron Wiggins,SG,CHO,13.929027,19.823744,14.5,7.548348
24,OKC,Jalen Williams,SG,CHO,28.715004,27.907798,29.5,23.079012
38,OKC,Isaiah Joe,SG,CHO,12.535180,14.932732,11.5,6.992913
41,OKC,Cason Wallace,SG,CHO,23.237501,24.073434,12.5,9.135382
29,OKC,Ajay Mitchell,SG,CHO,22.837864,24.116787,19.5,17.459282
71,OKC,Chet Holmgren,PF,CHO,27.904547,25.976065,27.5,26.682009


,Team,Player,Pos,Opp,MP,MP_L5_avg,PRA_line,PRA_proj
92,LAC,Ivica Zubac,C,GSW,24.060238,24.589258,26.5,18.976683
56,LAC,Kawhi Leonard,SF,GSW,37.327187,33.033447,40.5,44.619972
1,LAC,James Harden,PG,GSW,35.443169,32.378305,37.5,35.532238
10,LAC,Kris Dunn,PG,GSW,27.803558,27.733977,15.5,14.778962


...

,Team,Player,Pos,Opp,MP,MP_L5_avg,PRA_line,PRA_proj
34,ATL,Dyson Daniels,SG,TOR,35.289158,33.930738,25.5,29.743477
55,ATL,Jalen Johnson,SF,TOR,36.506443,34.642179,42.5,45.151794
95,ATL,Onyeka Okongwu,C,TOR,33.695900,33.411369,25.5,27.711950
19,ATL,Nickeil Alexander-Walker,SG,TOR,34.838127,33.648789,26.5,27.778364
90,ATL,Kristaps Porzingis,C,TOR,20.491772,24.190888,19.5,18.943569


,Team,Player,Pos,Opp,MP,MP_L5_avg,PRA_line,PRA_proj
83,TOR,Collin Murray-Boyles,PF,ATL,18.676691,23.619989,16.5,13.064535
54,TOR,RJ Barrett,SF,ATL,26.570839,27.547768,27.5,24.082541
97,TOR,Sandro Mamukelashvili,C,ATL,22.746017,23.528446,19.5,17.298922
9,TOR,Immanuel Quickley,PG,ATL,32.822514,33.526613,26.5,24.477247
14,TOR,Jamal Shead,PG,ATL,18.583153,24.783638,13.5,12.140724
74,TOR,Scottie Barnes,PF,ATL,35.425617,35.956332,34.5,35.666164
49,TOR,Brandon Ingram,SF,ATL,36.023071,34.390680,33.5,34.292694


,Team,Player,Pos,Opp,MP,MP_L5_avg,PRA_line,PRA_proj
85,GSW,Quinten Post,PF,LAC,17.151863,17.411929,12.5,9.455927
48,GSW,Jimmy Butler,SF,LAC,33.458622,31.185701,29.5,32.309326
84,GSW,Draymond Green,PF,LAC,22.098801,24.962959,18.5,16.304932
8,GSW,Stephen Curry,PG,LAC,34.862770,33.953576,35.5,34.361889
104,GSW,Al Horford,C,LAC,14.764151,14.767115,10.5,11.587261
28,GSW,Moses Moody,SG,LAC,21.217152,23.146505,13.5,12.447720
23,GSW,Brandin Podziemski,SG,LAC,25.675646,25.722337,17.5,18.394943


,Team,Player,Pos,Opp,MP,MP_L5_avg,PRA_line,PRA_proj
22,PHO,Devin Booker,SG,HOU,34.169140,32.817244,32.5,35.334934
13,PHO,Collin Gillespie,PG,HOU,29.823601,30.652685,20.5,22.400352
52,PHO,Dillon Brooks,SF,HOU,31.162651,29.637794,24.5,22.986418
30,PHO,Grayson Allen,SG,HOU,25.271309,28.755644,17.5,17.253405
64,PHO,Royce O'Neale,SF,HOU,29.074520,30.365114,14.5,14.459143


../tables/2025/gmday_preds_PRA.csv saved!
